In [28]:
# importing libraries
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,ToPILImage
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import cv2
from tqdm import tqdm
from sklearn.utils import shuffle
from skimage.io import imread, imshow
from tensorflow import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
from tensorflow.keras.utils import to_categorical
import keras.callbacks
from keras.callbacks import ModelCheckpoint
import random
from keras.applications import vgg16
from keras.preprocessing import image
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# Label and load the dataset

In [29]:
train_dir = 'C:/Users/Nitin/Downloads/other/image/Intel Image Classification/seg_train'
test_dir = 'C:/Users/Nitin/Downloads/other/image/Intel Image Classification/seg_test'
pred_dir = 'C:/Users/Nitin/Downloads/other/image/Intel Image Classification/seg_pred'

In [30]:
# assign class labels
class_names = ['buildings','forest','glacier','mountain','sea','street']
class_labels = {class_name:i for i, class_name in enumerate(class_names)}
print("class names with labels: ",class_labels)

number_classes = len(class_names)
print("number of classes: ", number_classes)

IMAGE_SIZE = (150,150)

class names with labels:  {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}
number of classes:  6


In [31]:
# usecase of os.listdir and os.path.join functions
for folder in os.listdir(train_dir):
    files_path = []
    for file in os.listdir(os.path.join(train_dir,folder)):
        files_path.append(file)
    print(folder,":", len(files_path))

buildings : 2191
forest : 2271
glacier : 2404
mountain : 2512
sea : 2274
street : 2382


In [32]:
# define load_dataset function to load dataset with labels
def load_dataset():
    # create list of datasets
    datasets = [train_dir, test_dir]
    output = []
    
    for dataset in datasets:
        
        images1 = []
        labels1 = []
        print(f"loading {dataset}")
        
        for folder in os.listdir(dataset):
            # assign labels to each folder images
            label = class_labels[folder]
            for file in tqdm(os.listdir(os.path.join(dataset,folder))):
                image_path = os.path.join(os.path.join(dataset, folder), file)
                # read the image files stored in image_path
                image_file = cv2.imread(image_path)
                image_file = cv2.cvtColor(image_file, cv2.COLOR_BGR2RGB)
                image_file = cv2.resize(image_file, IMAGE_SIZE)
                
                images1.append(image_file)
                labels1.append(label)
                
        # convert the images and labels list to numpy array
        images1 = np.array(images1, dtype = 'float32')
        labels1 = np.array(labels1, dtype = 'int32')
        
        output.append((images1, labels1))
        print("Images file have been loaded")
                
    return output 

In [33]:
# load the data using above functions
((train_images, train_labels), (test_images, test_labels)) = load_dataset()

loading C:/Users/Nitin/Downloads/other/image/Intel Image Classification/seg_train


100%|████████████████████████████████████████████████████████████████████████████| 2382/2382 [00:01<00:00, 1822.66it/s]


Images file have been loaded
loading C:/Users/Nitin/Downloads/other/image/Intel Image Classification/seg_test


100%|██████████████████████████████████████████████████████████████████████████████| 501/501 [00:00<00:00, 1834.98it/s]


Images file have been loaded


In [34]:
print("train dataset size:\n", "Images:",len(train_images), ", Labels", len(train_labels))
print("test dataset size:\n", "Images:",len(test_images), ", Labels", len(test_labels))


train dataset size:
 Images: 14034 , Labels 14034
test dataset size:
 Images: 3000 , Labels 3000


# Pre-process and explore image datasets

In [ ]:
# shape of image dataset is 4D tensors
train_images.shape, test_images.shape

In [ ]:
# shuffle the train nad test datasets
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)
test_images, test_labels = shuffle(test_images, test_labels, random_state=25)

In [ ]:
# keeping 6000 data instances for training and testing our model
train_images = train_images[:6000]
train_labels = train_labels[:6000]

test_images = test_images[:1400]
test_labels = test_labels[:1400]

In [ ]:
# let's see proportion of classes in train and test dataset
def proportion_labels(labels, dataset):
    series = pd.DataFrame(labels).reset_index()
    series.rename(columns = {0:'labels'}, inplace=True)
    final_s = series.groupby('labels').count().reset_index()                                          
    ticks1= [key for key in class_labels.keys()]
    
    # plot the pie chart and bar graph of labels
    plt.figure(figsize=(15,6))
    
    plt.subplot(121)
    plt.bar(np.array(final_s['labels']), np.array(final_s['index']))
    plt.xticks(ticks=np.array(final_s['labels']), labels=ticks1, fontsize=12, fontweight='bold')
    plt.yticks(fontsize=12, fontweight='bold')
    plt.grid(visible=True)
    plt.title("Number of images per class", size=14, weight='bold')
    
    plt.subplot(122)
    plt.pie(final_s['index'].ravel(),
            labels=ticks1,
            autopct='%1.2f%%',
            textprops={'fontweight':'bold'}
           )
    plt.title("proportion of classes", size=14, weight='bold')
    
    plt.suptitle(f"Proportion of {dataset} data", size=20, weight='bold')
    plt.show()
    
    return final_s

In [ ]:
final_s = proportion_labels(train_labels, "training")

In [ ]:
test_d = proportion_labels(test_labels, "test")

# Good practice: scale the data    

In [ ]:
# normalize the images dataset
trn_images_norm = train_images/255.0
tst_images_norm = test_images/255.0

In [ ]:
## plot 3*5 visualization of images
plt.figure(figsize=(15,9))
for idx, label in enumerate(train_labels[:20]):
    plt.subplot(4,5, idx+1)
    plt.imshow(trn_images_norm[idx])
    plt.xticks([])
    plt.yticks([])
    plt.title(f'image class: {class_names[label]}')   
    
plt.suptitle(f'Image samples from training data', size=20, weight='bold')
plt.show()

# Deep neural network model using CNN layers

Feature extraction process of CNN
The fundamental difference between a densely connected layer and a convolution layer is that, Dense layers learn global patterns in their input feature space, whereas convolution layers learn local patterns by small windows of feature map.

Feature extraction performed by CNN consists of 3 basic operations.

Filter: an image for a perticular feature (Convolution)
Detect: feature within the filtered image (ReLU)
Condense: image to enhance the features (Maxpooling)
Ref: Deep learning with Python by Francois Chollet, Kaggle learn's Convolution and relu chapter

# CNN2D layer arguments:

tf.keras.layers.Conv2D( filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), groups=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs )

Important arguments and their description

filters - Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution)

kernel_size - An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. Can be a single integer to specify the same value for all spatial dimensions

strides - An integer or tuple/list of 2 integers, specifying the strides of the convolution along the height and width. Can be a single integer to specify the same value for all spatial dimensions. Specifying any stride value != 1 is incompatible with specifying any dilation_rate value != 1.

padding - one of "valid" or "same" (case-insensitive). "valid" means no padding. "same" results in padding with zeros evenly to the left/right or up/down of the input. When padding="same" and strides=1, the output has the same size as the input.

activation - Activation function to use. If you don't specify anything, no activation is applied

In [ ]:
# model architecture of first keras model
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))
model.summary()

# Then, we can compile it with some parameters such as:

Optimizer: adam = RMSProp + Momentum. What is Momentum and RMSProp ?
Momentum = takes into account past gradient to have a better update.
RMSProp = exponentially weighted average of the squares of past gradients.
Loss function: we use sparse categorical crossentropy for classification, each images belongs to one class only

In [ ]:
# convert labels to categorical 
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# compile the model with 'adam' optimizer
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy']
             )

# SETUP A EARLY STOPPING CALL and model check point API
earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=5,
                                              verbose=1,
                                              mode='min'
                                              )
checkpointer = ModelCheckpoint(filepath='bestvalue', verbose=0, save_best_only=True)
callback_list = [checkpointer, earlystopping]

# fit model to the data
history = model.fit(trn_images_norm, train_labels, batch_size=128, epochs=15, validation_data=(tst_images_norm, test_labels),
                   callbacks=callback_list)

# evalute the model
test_loss, test_acc = model.evaluate(tst_images_norm, test_labels, verbose=0)
print("test loss and accuracy:", test_loss, test_acc)

# Note:

With 15 epochs and adam optimizer our test dataset accuracy comes around at 76-79% while our train accuracy stood at 80% at max.

This is the case of underfitting. to overcome underfitting, we need more deeper NN model to find highest accuracy. also we need to check where the error lies

# Plot the Loss and accuracy curves

In [ ]:
def plot_loss_accuracy(history):

    # create object of arrays of accuracy and loss
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # number of epochs in our model
    epochs = range(1 ,len(acc) + 1)
    
    # call matplolib figure object and plot loss and accuracy curves
    plt.figure(figsize=(15,6))
    
    plt.subplot(121)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title("Training and validation accuracy", fontsize=15)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel("accuracy", fontsize=14)
    plt.legend()
    
    plt.subplot(122)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title("Training and validation loss", fontsize=15)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel("loss", fontsize=14)
    plt.legend()
    
    plt.show()

# call the function to plot the curves
plot_loss_accuracy(history)

# Observation:
We can learn that image index numbers 6 and 7 are not predicted correctly from our prediction dataset. we will do error dataset on validation dataset

In [ ]:
from sklearn.metrics import confusion_matrix

CM = confusion_matrix(test_labels, pred_labels)
ax = plt.axes()
sn.heatmap(CM, annot=True, 
           annot_kws={"size": 10}, 
           xticklabels=class_names, 
           yticklabels=class_names, ax = ax)
ax.set_title('Confusion matrix')
plt.show()

# Error analysis on test data

In [ ]:
# prediction on validation data to check the errors
preds = model.predict(tst_images_norm)
print("Shape of the prediction indexs", preds.shape)

# apply argmax function over a rows to find 
predictions_f = np.argmax(preds, axis=1)
print("Shape of predicitons vector:", predictions_f.shape)

In [ ]:
# convert categorical test labels to numpy array of 1 dim
test_list = []
for i in test_labels:
    index1 = list(np.argwhere(i).reshape(1))[0]
    test_list.append(index1)

test_array = np.array(test_list)
print("shape of actual test array:", test_array.shape)

In [ ]:
def mislabled_images(class_names, test_images, test_labels, predicted_labels):
    # find the  mislabeled images using np.where and return array of mislabled images
    mislabels = (test_labels == predicted_labels)
    mislabels_index = np.array(np.where(mislabels == False)).ravel()
    mislables_images = test_images[mislabels_index]
    mislables_labels = predicted_labels[mislabels_index]
    
    print("Number of total mislabeled images", mislables_labels.shape)
    
    # plot the 20 mislabeled images
    plt.figure(figsize=(15,9))
    for idx, label in enumerate(mislables_labels[:20]):
        plt.subplot(4,5, idx+1)
        plt.imshow(mislables_images[idx])
        plt.xticks([])
        plt.yticks([])
        plt.title(f'Predicted class: {class_names[label]}')   
    
        plt.suptitle(f'Mislabeled images from test data', size=20, weight='bold')
    plt.show()  
    return mislables_labels

In [ ]:
mislables_labels = mislabled_images(class_names, tst_images_norm, test_array, predictions_f)

# Observations:

Model confuses between street with buildings and actual building class

Model confuses sea/glacier with mountains and actual mountains. model classifies them as mountains rathar than sea or glacier

It seems reasonable by seeing these variations that why these images are mislabeled. to mitigate this we have to build more deeper NN model with pre-trained base and increse accuracy of classification.

We won't do data augmentation in this problem, as these are the natural scenary images rather than specifics object images.

In [ ]:
# saving model in H5 format
#model.save('image_classifier_cnn1.h5')

# Prediction on unseen image data

In [ ]:
# load the prediction data and predict class on unseen data
def getImagePaths(path):
    image_names = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            fullpath = os.path.join(dirname, filename)
            image_names.append(fullpath)
    return image_names

images_paths = getImagePaths(pred_dir)
print("images_paths: ",len(images_paths))

# images path list to numpy array using cv2.imread module
file_array = []

for file in images_paths[:9]:
    files = cv2.imread(file)/255.0 
    file_array.append(files)
    
file_array = np.array(file_array)
# vector of prected probability of each class by our model
predictions = model.predict(file_array)
print("shape: ", predictions.shape)

# prediciton of 'index 0'image from 'preds_dir' image directory
preds_index = np.argmax(predictions, axis=1)
preds_index

In [ ]:
# sample prediciton on unseen data using model
plt.figure(figsize=(12,9))

for idx, i in enumerate(preds_index):
    plt.subplot(3,3, idx+1)
    plt.imshow(file_array[idx])
    plt.title(f"prediction of image: {class_names[i]}")
    plt.xticks([])
    plt.yticks([])
    
plt.suptitle("Samples predictions of model on unseen data", size=20, weight='bold')
plt.show()

# Predict labels of unseen data and classify them into their respective class

In [ ]:
def images_classifier(images_paths, class_names):
    file_array2 = []
    
    for file2 in images_paths:
        files2 = cv2.imread(file2)/255.0 
        file_array2.append(files2)
        
    file_array2 = np.array(file_array2)
    
    # run thorought the convolutional base and then model2
    img_arr = conv_base.predict(file_array2)
    
    # run img_arr of shape (9,4,4,512) through model2
    img_pred = model2.predict(img_arr)
    img_predf = np.argmax(img_pred, axis=1)
    print("shape of the predicted array:", img_predf.shape)
    
    # plot the 20 unseen images and predicted class
    plt.figure(figsize=(17,13))
    
    for idx, i in enumerate(img_predf):
        plt.subplot(4,5, idx+1)
        plt.imshow(file_array2[idx])
        plt.title(f"prediction of image: {class_names[i]}")
        plt.xticks([])
        plt.yticks([])
    
    plt.suptitle("Samples predicition of model2 on unseen data", size=20, weight='bold')
    plt.show()
    
images_classifier(images_paths[:20], class_names)

# Observation:

Here, we can see how model2 using vgg16 as a base performed really well than previous model. model2 could correctly classify every image of first 20 samples.

It improved from previous errors of buildings/street and mountain/glacier confusions.

In [ ]:
# saving model2 with h5 extension
#model2.save("image_classifier_cnn2.h5")

# Conclusions:

In this notebook, We built 2 Deep Neural network models using CNN layers.

Our first model's accuracy was at around 78% and our second model's accuracy improved to 87% which is significant in our problem.

We saw workings of CNN layer with key parameters and their description. we also learned about VGG16 pretrained base and its architecture.
Finally, we visualized intermediate activations of each layer of model1.

In [ ]:
https://www.kaggle.com/code/avikumart/computervision-intel-image-classification-project
https://www.kaggle.com/code/vincee/intel-image-classification-cnn-keras